In [11]:
import open3d as o3d
import numpy as np
import os
import glob
import pandas as pd
import pymeshlab as pml
import time
import matplotlib as plt
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [38]:
def shift_mesh(mesh, distance):
    "shift all values of mesh to the right to display meshes next to eachother"
    mesh.vertices = o3d.utility.Vector3dVector(np.asarray(mesh.vertices) + np.array([0, distance, 0]) )
    return mesh

def draw_meshes_results(result_list, draw_unit_cube=False, draw_coordinate_frame=False):
    """Draws a list of given meshes."""
    value = 0
    mesh_list = []
    for i in result_list:
        mesh1 = o3d.io.read_triangle_mesh(i+".obj")
        mesh1 = shift_mesh(mesh1, value)
        mesh_list.append(mesh1)
        value += 1
    o3d.visualization.draw_geometries(mesh_list, mesh_show_wireframe = True)

In [24]:
"matching function (can be replaced by improved matching function)"

#computes euclidean distance between 2 vectors
def euclidean_distance(vector1, vector2):
    euclidean = np.linalg.norm(np.asarray(vector1[2:9]) - np.asarray(vector2[2:9]))
    return euclidean

#computes cosine distance between 2 vectors
def cosine_distance(vector1, vector2):
    one_arr = np.asarray(vector1)
    two_arr = np.asarray(vector2)
    cosine = 1 - ((one_arr @ two_arr) / (np.linalg.norm(one_arr) * np.linalg.norm(two_arr)))
    return cosine

def scale_dict(my_dict, max_val):
    for i in my_dict:
        my_dict[i] = float(my_dict[i]/max_val)
    return my_dict 

#returns top k matches
def match(vector1, k):
    distance_dictionary = {}
    distance_dictionary_2 = {}
    for i in range(len(data)):
        vector2 = data.iloc[i]
        match_path = vector2[1]
        distance = cosine_distance(vector1[2:9], vector2[2:9])

        distance_dictionary[match_path] = distance

        distance_2 = euclidean_distance(vector1[2:9], vector2[2:9])
        distance_dictionary_2[match_path] = distance_2

    #combining the two distance dicts
    max_value = max(distance_dictionary_2, key=distance_dictionary_2.get)
    maximum = distance_dictionary_2[max_value]
    distance_dictionary_2 = scale_dict(distance_dictionary_2, maximum)
    

    # Sort the dictionary by value using a lambda function to extract the values
    result = dict(sorted(distance_dictionary.items(), key = lambda x: x[1], reverse = False)[:k])
    return result

In [36]:

def vis(input_object, k):
    "match input onject using distance function -> make list of top k similar items -> display top k similar items"
    result_list = []
    print("input object path:", input_object[1])
    result = match(input_object, k)
    for path in result:
        dbpath = r"./resampledO3D/"+ path.split("/")[2]
        result_list.append(dbpath)
    draw_meshes_results(result_list)

In [53]:
data = pd.read_pickle("normalized_features_final.pkl")
vector1 = data.iloc[2]
print(type(vector1))
test_vec = "./database/AircraftBuoyant\m1337.obj"

path = "./features/"+test_vec.split("/")[2].split(".")[0]

value = data.loc[data["path"] == path]

print(value.squeeze())

<class 'pandas.core.series.Series'>
class                                      AircraftBuoyant
path                      ./features/AircraftBuoyant\m1337
V                                                 4.100846
S                                                 0.214806
c                                                -0.021151
D                                                -0.431171
R                                                 1.524371
E                                                 4.945361
C                                                 1.790998
A3       [0.000719, 0.002245, 0.003637, 0.004885, 0.006...
D1       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
D2       [0.0, 0.00048, 0.00051, 0.00104, 0.00095, 0.00...
D3       [1e-06, 5e-06, 1.9e-05, 5.4e-05, 0.000111, 0.0...
D4       [1e-05, 3.7e-05, 0.000109, 0.000236, 0.000379,...
Name: 1, dtype: object


In [39]:
print(data)

              class                              path         V         S  \
1   AircraftBuoyant  ./features/AircraftBuoyant\m1337  4.100846  0.214806   
1   AircraftBuoyant  ./features/AircraftBuoyant\m1338  4.728118  0.540607   
1   AircraftBuoyant  ./features/AircraftBuoyant\m1339  0.087864  0.155798   
1   AircraftBuoyant  ./features/AircraftBuoyant\m1340  0.219291 -0.646455   
1   AircraftBuoyant  ./features/AircraftBuoyant\m1341 -0.458655 -0.605102   
..              ...                               ...       ...       ...   
1        WheelChair      ./features/WheelChair\D00687 -0.257985 -0.136291   
1        WheelChair      ./features/WheelChair\D00720  0.415856 -0.265226   
1        WheelChair      ./features/WheelChair\D00782  0.330954 -0.270341   
1        WheelChair      ./features/WheelChair\D00821  0.315740 -0.136219   
1        WheelChair      ./features/WheelChair\D00846  0.174475 -0.376880   

           c         D         R         E         C  \
1  -0.021151 -0.431